In [12]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

In [29]:
df = pd.read_csv('file/df_health.csv')
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('category01', axis=1)
df.head()

,category02,question,answer
0,33.다음 진료실 예약,이동훈 도수 치료사분께 도수 치료 진행 받고 싶어요. 가장 빠른 일정을 안내해 주시...,현재 예약 가능한 가장 빠른 일정으로는 9월 4일 열한시가 가능합니다.
1,33.다음 진료실 예약,박진우 원장님 진료 예약 가능 일정을 조회해 주세요. 최대한 빠른 일정이 언제인가요?,"가장 빠른 예약 가능 일정으로는 9월 12일 열시, 13일 열한시 가능합니다."
2,30.약국 위치 문의,1층 약국은 언제 리모델링이 끝나나요? 여기로 오려니 불편하네요.,다음주 월요일부터 정상 영업에 들어갑니다. 양해 부탁드리겠습니다.
3,28.증상 상담,증상을 지금 다 말해야 돼요? 그냥 요즘 너무 힘들어요.,의사 선생님이 오시면 환자분의 증상에 대한 자세한 상담을 받으실 수 있습니다.
4,31.수납 방법 안내,결재할게요. 약값 포함해서 얼마인가요?,약은 약국에서 처방 받으신 뒤 약국에서 결제하시면 됩니다. 진료비와 수액 처방을 포...


In [30]:
# 정규표현식을 사용하여 카테고리 앞에 있는 숫자와 점을 제거
df['category02'] = df['category02'].str.replace(r'^\d+\.', '', regex=True)

# '진료 접수 안내'와 '진료 접수'를 같은 카테고리로 변경
# df['category02'] = df['category02'].replace({'진료 접수': '진료 접수 안내'})

# 고유한 카테고리 값 추출
unique_categories = df['category02'].unique()
print(unique_categories)

['다음 진료실 예약' '약국 위치 문의' '증상 상담' '수납 방법 안내' '진료 접수 안내' '증빙 서류 발급'
 '대기실 및 진료실 위치 안내']


In [38]:
def replace_content(value):
    replacements = {
        '예약': '예약',
        '약국': '약국',
        '증상': '증상',
        '수납': '수납',
        '진료 접수': '진료접수',
        '증빙 서류': '증빙서류',
        '위치': '위치'
    }
    for key, new_value in replacements.items():
        if key in value:
            return new_value
    return value

# 'Document' 열의 내용을 변경
df['category02'] = df['category02'].apply(replace_content)
# 고유한 카테고리 값 추출
unique_categories = df['category02'].unique()
print(unique_categories)
df.to_csv('file/df_health.csv', index=False)

['예약' '약국' '증상' '수납' '진료접수' '증빙서류' '위치']


In [33]:
df.

,category02,question,answer
count,27531,27531,27531
unique,7,24400,8919
top,진료접수,대면으로 수납하려고 하는데 대기표 주세요.,옆에 보이시는 메뉴에서 서류 발급 버튼 누르시면 다양한 서류들을 발급받으실 수 있습니다.
freq,3981,13,29


In [36]:
filtered_data = df[df['question'].str.contains('정수기', na=False)]
filtered_data

,category02,question,answer
103,위치,목이 마른데 로비에 정수기가 없나요? 어디로 가서 물을 마실 수 있죠?,대기실에 정수기가 있습니다. 저를 따라오시면 대기실까지 안내 도와드리겠습니다.
1705,위치,"아 목마른데, 로비에 정수기 없어요? 어디로 가서 물을 마실 수 있는 거예요?",대기실에 정수기가 있습니다. 저를 따라오시면 대기실까지 안내 도와드리겠습니다.
3282,위치,"물을 마시고 싶은데, 혹시 정수기도 있을까요?",정수기는 진료 대기실 앞쪽에 있습니다. 감사합니다.
3537,위치,목이 마른데 로비에 정수기가 없어요? 물은 어디서 마셔야 돼요?,대기실에 정수기가 있습니다. 저를 따라오시면 대기실까지 안내 도와드리겠습니다.
3794,위치,"너무 목이 마른데요, 혹시 정수기 같은 건 없나요?",정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.
4146,위치,목이 너무 마른데요. 대기실에 정수기가 안 보여요. 정수기 어디에 있나요?,정수기는 접수처 방면에서 왼편에 위치해있습니다.
5038,위치,"목이 마른데, 로비에 정수기가 없어요? 물 마시려면 어디로 가야 돼요?",대기실에 정수기가 있습니다. 저를 따라오시면 대기실까지 안내 도와드리겠습니다.
8784,위치,"물을 마시고 싶은데, 정수기를 못 찾겠어요. 어디에 있어요?",정수기는 진료 대기실 앞쪽에 있습니다. 감사합니다.
8892,위치,너무 목이 말라서 물 마시고 싶은데 정수기 있을까요?,정수기는 진료 대기실 앞쪽에 있습니다. 감사합니다.
9491,위치,물을 마시고 싶은데 혹시 정수기는 어디에 있나요?,정수기는 진료 대기실 앞쪽에 있습니다. 감사합니다.


In [37]:
# 고유한 카테고리 값 추출
unique_categories = filtered_data['category02'].unique()
print(unique_categories)

['위치']


In [6]:
question = df['question'].tolist()
question[:5]

['너무 목이 마른데요, 혹시 정수기 같은 건 없나요?',
 '교정기가 떨어진 것 같아서 붙이러 왔어요.',
 '앞으로 얼마나 나아지는지 보고 아프면 오라고 하네요. 제가 다음에 예약하고 와도 되죠?',
 '이것도 수납용 키오스크인가요? 대기 줄이 너무 길어서요.',
 '주사실 안에 가서 있으라는데 주사실이 어딘데요?']